In [3]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader


/home/victor/Documents/travail_epita/venv_epita/lib/python3.10/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
url="https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/dbpedia-entity.zip"
data_path = util.download_and_unzip(url,"./data")
corpus,queries,qrels = GenericDataLoader(data_folder=data_path).load(split="test")

  0%|          | 0/4635922 [00:00<?, ?it/s]

In [5]:
len(corpus)

4635922

In [6]:
queries

{'INEX_LD-2009022': 'Szechwan dish food cuisine',
 'INEX_LD-2009039': 'roman architecture',
 'INEX_LD-2009053': 'finland car industry manufacturer saab sisu',
 'INEX_LD-2009061': 'france second world war normandy',
 'INEX_LD-2009062': 'social network group selection',
 'INEX_LD-2009063': 'D-Day normandy invasion',
 'INEX_LD-2009074': 'web ranking scoring algorithm',
 'INEX_LD-2009115': 'virtual museums',
 'INEX_LD-2010004': 'Indian food',
 'INEX_LD-2010014': 'composer museum',
 'INEX_LD-2010019': 'gallo roman architecture in paris',
 'INEX_LD-2010020': 'electricity source in France',
 'INEX_LD-2010037': 'social network API',
 'INEX_LD-2010043': 'List of films from the surrealist category',
 'INEX_LD-2010057': 'Einstein Relativity theory',
 'INEX_LD-2010069': 'summer flowers',
 'INEX_LD-2010100': 'house concrete wood',
 'INEX_LD-2010106': 'organic food advantages disadvantages',
 'INEX_LD-20120111': 'vietnam war movie',
 'INEX_LD-20120121': 'vietnam food recipes',
 'INEX_LD-20120122': '

In [7]:
corpus["<dbpedia:ASCII>"]

{'text': 'ASCII (/ˈæski/ ASS-kee), abbreviated from American Standard Code for Information Interchange, is a character-encoding scheme (the IANA prefers the name US-ASCII).  ASCII codes represent text in computers, communications equipment, and other devices that use text. Most modern character-encoding schemes are based on ASCII, though they support many additional characters.',
 'title': 'ASCII'}